In [ ]:
!pip install pandas
!pip install spacy==3.2.4
!pip install plac

In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [24]:
!curl  https://www.dropbox.com/s/sukruo2u3tiq5fr/punct-data.zip?dl=1  punct_data.zip

301 Moved Permanently
The resource has been moved to /s/dl/sukruo2u3tiq5fr/punct-data.zip;
you should be redirected automatically.




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   138    0   138    0     0    282      0 --:--:-- --:--:-- --:--:--   282
100   138    0   138    0     0    281      0 --:--:-- --:--:-- --:--:--   282
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: punct_data.zip


In [11]:
!rm -r punct-data/

'rm' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [25]:
!tar -xf punct-data.zip

In [32]:
from collections import Counter
import pandas as pd
import re 
from collections import defaultdict
from spacy.matcher import Matcher
from spacy.tokens import Span
from tqdm import tqdm
import spacy

In [33]:
nlp = spacy.blank('pt')

In [36]:
def preprocess_data(dataframe):
     

    TOTAL = len(list(dataframe.groupby("sentence_id")))
    data = []
    for _, group in tqdm(dataframe.groupby("sentence_id"), total=TOTAL):
        text = " ".join(group.words)
        doc = nlp.make_doc(text)
        ents = []
        count_dict = defaultdict(lambda : 0)
        for i, label in enumerate(group.labels):
            if label != "O":
                matching_word = group.words.tolist()[i]
                span = Span(doc, i, i+1, label=label)
                ents.append((span.start_char, span.end_char, label.replace('I-','')))
                count_dict[matching_word] += 1
        ent = (text, {
            'entities': ents
        })
        data.append(ent)
    return data

100%|██████████| 4072/4072 [00:04<00:00, 910.13it/s] 


In [38]:
import pickle

import pickle
train_df = pd.read_csv('./punct-data/train.csv').dropna()
TRAIN_DATA = preprocess_data(train_df)


with open('punct.pickle', 'wb') as handle:
    pickle.dump(TRAIN_DATA, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
with open('filename.pickle', 'rb') as handle:
    TRAIN_DATA = pickle.load(handle)
TRAIN_DATA[:1]

[('filmado na cidade de nova york exibido pela primeira vez no ted2018 sou um imigrante da uganda vivendo nos estados unidos enquanto espero a aprovação do meu pedido de asilo',
  {'entities': [(167, 172, 'PERIOD')]})]

In [ ]:
train_df.loc[train_df['sentence_id']==0,]

,sentence_id,labels,words
0,0,O,filmado
1,0,O,na
2,0,O,cidade
3,0,O,de
4,0,O,nova
5,0,O,york
6,0,O,exibido
7,0,O,pela
8,0,O,primeira
9,0,O,vez


In [ ]:
TRAIN_DATA[2:3]

[('mas também demonstra o que costuma faltar no debate global sobre refugiados migrantes e imigrantes as vozes daqueles desfavorecidos os cidadãos de muitos países anfitriões até mesmo daqueles que acolhiam os recém chegados estão preocupados com o aumento do número de indivíduos chegando aos seus países a primeira crítica é que os recém chegados prejudicam a estabilidade do bem estar social e dos níveis de emprego em seus países cidadãos inseguros e céticos olham para os políticos que competem entre si para ver quem ganha o prêmio de voz mais alta do populismo e do nacionalismo é uma competição entre quem é o mais duro com os migrantes quem é mais propenso a impor proibições de viagem e quem é o mais ansioso em propor projetos de construção de muros',
  {'entities': [(65, 75, 'COMMA'),
    (88, 98, 'COMMA'),
    (117, 131, 'PERIOD'),
    (161, 171, 'COMMA'),
    (213, 221, 'COMMA'),
    (296, 302, 'PERIOD'),
    (424, 430, 'PERIOD'),
    (570, 582, 'PERIOD'),
    (632, 641, 'COMMA'),


In [ ]:
TRAIN_DATA[:1]

[('filmado na cidade de nova york exibido pela primeira vez no ted2018 sou um imigrante da uganda vivendo nos estados unidos enquanto espero a aprovação do meu pedido de asilo',
  {'entities': [(167, 172, 'PERIOD')]})]

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example
#import spacy_transformers


def train_spacy(data, iterations=10, model="pt_core_news_md"):
    TRAIN_DATA = data

    # nlp = spacy.blank("en")  # start with a blank model

    if model is not None:
      nlp = spacy.load(model)
      print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('pt')  
        print("Created blank 'pt' model")

    #set up the pipeline

    if 'ner' not in nlp.pipe_names:
        #ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner', last=True)
    else:
        ner = nlp.get_pipe('ner')
    

    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Init loss
    losses = None

    # Init and configure optimizer
    optimizer = nlp.create_optimizer()
    optimizer.learn_rate = 0.001  # Change to some lr you prefers
    batch_size = 32  # Choose batch size you prefers

    for itn in range(iterations):
        print("Starting iterations " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}

        # Batch the examples and iterate over them
        for batch in spacy.util.minibatch(TRAIN_DATA, size=batch_size):
            # Create Example instance for each training example in mini batch
            #print(annotations)
            examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]
            # Update model with mini batch
            
            losses = nlp.update(examples, drop=0.2, sgd=optimizer)

    print(losses)
    return nlp

Loaded model 'pt_core_news_md'
Starting iterations 0


In [ ]:
test_df = pd.read_csv('./punct-data/train.csv').dropna()
label2dict = {"COMMA":",","PERIOD": ".", "QUESTION":"?"}
TOTAL = len(list(test_df.groupby("sentence_id")))

In [ ]:

examples = []
for text, entities in TEST_DATA:
    doc = nlp(text)
    
    #print(doc.ents[0].start_char,doc.ents[0].end)
    #print(doc.char_span(ent.start, ent.end))
    
    example = Example.from_dict(doc, entities)
    examples.append(example)
    
    #print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
from spacy.scorer import Scorer
scorer = Scorer(nlp)
scores = Scorer.score_spans(examples, "ents")
print(scores["ents_f"])

In [ ]:
!zip ner.zip -r ner 

In [ ]:
scores

In [ ]:
output_dir = 'ner'
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)